In [52]:
import numpy as np
import pandas as pd
import datetime as dt
import warnings 
warnings.filterwarnings('ignore')

In [2]:
data=pd.read_csv("D:/Hackathon/ad_clicks/train.csv",parse_dates=True)

In [14]:
data1=data

In [15]:
data1.head()

,ID,datetime,siteid,offerid,category,merchant,countrycode,browserid,devid,click,hour,weekday
0,IDsrk7SoW,2017-01-14 09:42:09,4709696.0,887235,17714,20301556,4,2,-1,0,9,5
1,IDmMSxHur,2017-01-18 17:50:53,5189467.0,178235,21407,9434818,1,8,0,0,17,2
2,IDVLNN0Ut,2017-01-11 12:46:49,98480.0,518539,25085,2050923,0,1,-1,0,12,2
3,ID32T6wwQ,2017-01-17 10:18:43,8896401.0,390352,40339,72089744,2,2,1,0,10,1
4,IDqUShzMg,2017-01-14 16:02:33,5635120.0,472937,12052,39507200,3,8,0,0,16,5


In [100]:
type(data['datetime'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [4]:
data['datetime']=pd.to_datetime(data['datetime'])

In [1]:
data.corr()

In [5]:
data['month'],data['year']=data['datetime'].dt.month,data['datetime'].dt.year

In [2]:
data['day'].value_counts()

In [3]:
data['datetime'].nunique()

In [6]:
data.drop(['month','year'],axis=1,inplace=True)

In [7]:
data['day'],data['hour']=data['datetime'].dt.day,data['datetime'].dt.hour

In [8]:
data['weekday']=data['datetime'].dt.weekday

In [42]:
data=data.dropna()

In [4]:
data.info()

In [9]:
data["countrycode"] = data['countrycode'].astype('category')
data["browserid"] = data['browserid'].astype('category')
data["devid"] = data['devid'].astype('category')

In [10]:
data["browserid"] = data['browserid'].cat.codes
data["devid"] = data['devid'].cat.codes

In [11]:
data["countrycode"] = data['countrycode'].cat.codes

In [5]:
#data.head()

In [6]:
#data['click'].value_counts()

In [13]:
data.drop(['day'],axis=1,inplace=True)

In [75]:
X=data.drop(['click'],axis=1)
X=X[['countrycode','browserid','devid','hour','weekday']]
y=data.click

In [76]:
X.columns

Index(['countrycode', 'browserid', 'devid', 'hour', 'weekday'], dtype='object')

In [72]:
X.drop(['ID','datetime'],axis=1,inplace=True)

In [60]:
test=pd.read_csv("D:/Hackathon/ad_clicks/test.csv",parse_dates=True)

In [61]:
test=test.dropna()

In [7]:
test['datetime']=pd.to_datetime(test['datetime'])
test['hour']=test['datetime'].dt.hour
test['weekday']=test['datetime'].dt.weekday
test["countrycode"] = test['countrycode'].astype('category')
test["browserid"] = test['browserid'].astype('category')
test["devid"] = test['devid'].astype('category')
test["browserid"] = test['browserid'].cat.codes
test["devid"] = test['devid'].cat.codes
test["countrycode"] = test['countrycode'].cat.codes
test.head()

In [69]:
test.drop(['ID','datetime'],axis=1,inplace=True)

In [66]:
X_test

8821613

In [8]:
#data.info()

In [73]:
from sklearn.model_selection import train_test_split

In [81]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42,stratify=y)

In [9]:
#y_train.value_counts()

In [46]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression(class_weight={0:0.3,1:0.7})
logreg.fit(X_train,y_train)
threshold=0.30
y_predict=np.where(logreg.predict_proba(X_test)[:,1]>threshold,1,0)

In [47]:
# import packages
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Code starts here
cf=confusion_matrix(y_test,y_predict)
acc=accuracy_score(y_test,y_predict)
precision=precision_score(y_test,y_predict)
recall=recall_score(y_test,y_predict)
f_score=f1_score(y_test,y_predict)


print(cf)
print(acc)
print(precision)
print(recall)
print(f1_score)
print(acc)


# Code starts here

[[3458932   51247]
 [  38040   93124]]
0.9754796513264474
0.6450325896475054
0.7099813973346345
<function f1_score at 0x000000FAE34640D0>
0.9754796513264474


In [48]:
# import packages
from sklearn.metrics import roc_auc_score

roc=roc_auc_score(y_test,y_predict)

print(roc)


0.8476909284846571


In [49]:
f1_score(y_test,y_predict)

0.6759504237211243

In [91]:
data.columns

Index(['ID', 'datetime', 'siteid', 'offerid', 'category', 'merchant',
       'countrycode', 'browserid', 'devid', 'click', 'hour', 'weekday'],
      dtype='object')

In [10]:
data['siteid']=data['siteid'].astype('int64')

In [11]:
from sklearn.preprocessing import StandardScaler


In [131]:
data_encoding=data1

In [12]:
#data_encoding.head()

In [13]:
#data_encoding.columns

In [133]:
data_encoding=data_encoding[['countrycode','browserid','devid','day','hour','weekday','click']]

In [134]:
data_encoding=data_encoding.dropna()

In [135]:
X=data_encoding.drop(['click'],axis=1)
y=data_encoding.click

In [136]:
print(X.columns)

Index(['countrycode', 'browserid', 'devid', 'day', 'hour', 'weekday'], dtype='object')


In [137]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42,stratify=y)

In [141]:
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, tst_series=None, target=None, min_samples_leaf=1,smoothing=1,noise_level=0):
    #assert len(trn_series) == len(target)
    #assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(trn_series.to_frame(trn_series.name),
                             averages.reset_index().rename(columns={'index': target.name, target.name: 'average'})
                             ,on=trn_series.name,how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(tst_series.to_frame(tst_series.name),
                             averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
                             on=tst_series.name,how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [151]:
trn, sub = target_encode(X_train["browserid"], X_test["browserid"],
                         target=y_train, min_samples_leaf=100,smoothing=10,noise_level=0.01)


In [154]:
trn1, sub1 = target_encode(X_train["countrycode"], X_test["countrycode"],
                         target=y_train, min_samples_leaf=100,smoothing=10,noise_level=0.01)


In [159]:
trn2, sub2 = target_encode(X_train["devid"], X_test["devid"],
                         target=y_train, min_samples_leaf=100,smoothing=10,noise_level=0.01)


In [14]:
trn2

In [15]:
X_train['te_countrycode']=trn
X_test['te_countrycode']=sub

In [158]:
X_train['te_browserid']=trn1
X_test['te_browserid']=sub1

In [162]:
X_train['te_devid']=trn2
X_test['te_devid']=sub2

In [165]:
X_train.drop(['browserid','devid','countrycode'],axis=1,inplace=True)
X_test.drop(['browserid','devid','countrycode'],axis=1,inplace=True)

In [16]:
X_train.drop(['day'],axis=1,inplace=True)#X_test.drop(['day'],axis=1,inplace=True)
#X_test.head()

In [41]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression(class_weight={0:0.1,1:0.9})
logreg.fit(X_train,y_train)
threshold=0.25
y_predict=np.where(logreg.predict_proba(X_test)[:,1]>threshold,1,0)

In [42]:
# import packages
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Code starts here
cf=confusion_matrix(y_test,y_predict)
acc=accuracy_score(y_test,y_predict)
precision=precision_score(y_test,y_predict)
recall=recall_score(y_test,y_predict)
f_score=f1_score(y_test,y_predict)


print(cf)
print(acc)
print(precision)
print(recall)
print(f1_score)
print(acc)


# Code starts here

[[3084110  426069]
 [  12310  118854]]
0.8796106271779396
0.2181115497051877
0.906148028422433
<function f1_score at 0x000000FAE34640D0>
0.8796106271779396


In [43]:
# import packages
from sklearn.metrics import roc_auc_score

roc=roc_auc_score(y_test,y_predict)

print(roc)


0.8923835195099492


In [45]:
f1_score(y_test,y_predict)

0.35159380375602545

In [172]:
data.columns

Index(['ID', 'datetime', 'siteid', 'offerid', 'category', 'merchant',
       'countrycode', 'browserid', 'devid', 'click', 'hour', 'weekday'],
      dtype='object')

In [177]:
data['merchant'].value_counts()

99510044    328721
43452411    274991
74363610    263258
26391770    262966
49384126    177525
57977218    166509
555603      166460
7122654     160218
29916445    147510
74868704    147246
7864011     142589
31388981    136416
27723861    136269
20301556    135844
43770446    131234
91553313    129632
30860214    128050
65017662    115660
37135965    113479
15961717    109215
4000296     105422
47079934    103136
87807327     97733
55358028     90557
67290861     88762
61940782     87014
67147700     85042
2050923      80061
54526992     78172
93157824     77234
             ...  
82396611        38
68558271        36
61673872        35
53065903        34
44247871        33
90293513        33
79649944        31
4571619         29
71136727        29
74599462        29
23020204        28
15946833        28
78629268        28
39714470        28
38069694        23
1042419         22
38685003        19
32812292        18
8148112         18
77041230        17
34742359        16
51795179    

In [176]:
data['category'].value_counts()

40339    541247
42227    501017
6906     445006
904      294707
17714    275308
82877    217625
5602     183605
97863    183362
1678     176132
11837    174392
48498    173051
80554    157987
34670    142544
41706    137433
73248    136770
33638    129676
10345    129661
12211    119507
95113    115750
45620    114474
54916    105819
92704    101930
9764     100825
30580     96049
93286     82413
21407     82131
82396     80192
53956     80081
8467      77770
23576     76085
          ...  
43133       611
62180       554
52453       497
76126       395
55206       387
25259       367
48797       332
2757        324
12019       208
39112       206
38881       205
86014       197
56150       193
7353        182
84407       148
97          145
47190       135
277         130
35380       129
91868       124
18243       118
77227       114
17801       112
19010       100
25959        98
88170        75
20111        67
21503        42
51532        16
69325         8
Name: category, Length: 

In [17]:
data_encoding1=data1
data_encoding1=data_encoding1[['category','merchant','countrycode','browserid','devid','hour','weekday','click']]





In [17]:
data_encoding1['daytime'] = pd.cut(data_encoding1['hour'],[1,7,13,19,25],labels=['night','morning','afternoon','evening'])

In [18]:
#data_encoding1.head()

In [19]:
data_encoding1.dropna()

In [20]:
data_encoding1.drop(['hour'],axis=1,inplace=True)

In [22]:
X=data_encoding1.drop(['click'],axis=1)
y=data_encoding1.click

In [23]:
X.columns

Index(['category', 'merchant', 'countrycode', 'browserid', 'devid', 'weekday',
       'daytime'],
      dtype='object')

In [25]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42,stratify=y)

In [26]:
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, tst_series=None, target=None, min_samples_leaf=1,smoothing=1,noise_level=0):
    #assert len(trn_series) == len(target)
    #assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(trn_series.to_frame(trn_series.name),
                             averages.reset_index().rename(columns={'index': target.name, target.name: 'average'})
                             ,on=trn_series.name,how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(tst_series.to_frame(tst_series.name),
                             averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
                             on=tst_series.name,how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)




In [212]:
X_train.columns

Index(['category', 'merchant', 'countrycode', 'browserid', 'devid', 'day',
       'weekday', 'daytime', 'en_category', 'en_merchant', 'en_countrycode',
       'en_browserid', 'en_devid', 'en_day', 'en_weekday', 'en_daytime'],
      dtype='object')

In [27]:
trn, sub = target_encode(X_train["category"], X_test["category"],
                         target=y_train, min_samples_leaf=100,smoothing=10,noise_level=0.01)


In [28]:
trn1, sub1 = target_encode(X_train["merchant"], X_test["merchant"],
                         target=y_train, min_samples_leaf=100,smoothing=10,noise_level=0.01)


In [29]:
trn2, sub2 = target_encode(X_train["countrycode"], X_test["countrycode"],
                         target=y_train, min_samples_leaf=100,smoothing=10,noise_level=0.01)


In [30]:
trn3, sub3 = target_encode(X_train["browserid"], X_test["browserid"],
                         target=y_train, min_samples_leaf=100,smoothing=10,noise_level=0.01)


In [31]:
trn4, sub4 = target_encode(X_train["devid"], X_test['devid'],
                         target=y_train, min_samples_leaf=100,smoothing=10,noise_level=0.01)


In [32]:
trn6, sub6 = target_encode(X_train["weekday"], X_test["weekday"],
                         target=y_train, min_samples_leaf=100,smoothing=10,noise_level=0.01)


In [33]:
trn7, sub7 = target_encode(X_train["daytime"], X_test["daytime"],
                         target=y_train, min_samples_leaf=100,smoothing=10,noise_level=0.01)


In [21]:
X_train['en_category']=trn

In [213]:
X_train.columns

Index(['category', 'merchant', 'countrycode', 'browserid', 'devid', 'day',
       'weekday', 'daytime', 'en_category', 'en_merchant', 'en_countrycode',
       'en_browserid', 'en_devid', 'en_day', 'en_weekday', 'en_daytime'],
      dtype='object')

In [22]:
X_train['en_merchant']=trn1
X_train['en_countrycode']=trn2
X_train['en_browserid']=trn3
X_train['en_devid']=trn4

X_train['en_weekday']=trn6
X_train['en_daytime']=trn7

In [23]:
X_test['en_category']=sub
X_test['en_merchant']=sub1
X_test['en_countrycode']=sub2
X_test['en_browserid']=sub3
X_test['en_devid']=sub4

X_test['en_weekday']=sub6
X_test['en_daytime']=sub7

In [24]:
X_train.drop(['category', 'merchant', 'countrycode', 'browserid', 'devid','weekday', 'daytime'],axis=1,inplace=True)


In [25]:
X_test.drop(['category', 'merchant', 'countrycode', 'browserid', 'devid','weekday', 'daytime'],axis=1,inplace=True)

In [40]:
X_train.columns

Index(['en_category', 'en_merchant', 'en_countrycode', 'en_browserid',
       'en_devid', 'en_weekday', 'en_daytime'],
      dtype='object')

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression(class_weight={0:0.3,1:0.7})
logreg.fit(X_train,y_train)
threshold=0.30
y_predict=np.where(logreg.predict_proba(X_test)[:,1]>threshold,1,0)

In [ ]:
# import packages
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Code starts here
cf=confusion_matrix(y_test,y_predict)
acc=accuracy_score(y_test,y_predict)
precision=precision_score(y_test,y_predict)
recall=recall_score(y_test,y_predict)
f_score=f1_score(y_test,y_predict)


print(cf)
print(acc)
print(precision)
print(recall)
print(f1_score)
print(acc)


# Code starts here

In [53]:
X_train.columns

Index(['en_category', 'en_merchant', 'en_countrycode', 'en_browserid',
       'en_devid', 'en_weekday', 'en_daytime'],
      dtype='object')

In [69]:
import pandas as pd
import numpy as np
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest, f_classif
import re
import matplotlib.pyplot as plt


#https://www.kaggle.com/dhey24/ensemble-of-logistic-and-gradient-boosted








predictors = ['en_category', 'en_merchant', 'en_countrycode', 'en_browserid','en_devid', 'en_weekday', 'en_daytime']

# Perform feature selection
selector = SelectKBest(f_classif, k=5)
selector.fit(train[predictors], train['click'])

# Get the raw p-values for each feature, and transform from p-values into scores
#scores = -np.log10(selector.pvalues_)

# Plot the scores.  See how "Pclass", "Sex", "Title", and "Fare" are the best?
#plt.bar(range(len(predictors)), scores)
#plt.xticks(range(len(predictors)), predictors, rotation='vertical')
#plt.show()

#use top 4
#predictors = ["Pclass", "Sex", "Fare", "Title"]

#Ensemble Gradient Boosting and Logistic Regression
algorithms = [[GradientBoostingClassifier(random_state=1, n_estimators=25, max_depth=3),
               [ 'en_countrycode', 'en_browserid','en_devid',  'en_daytime']],
              [LogisticRegression(random_state=1),
               ['en_category', 'en_merchant',  'en_browserid','en_devid', 'en_daytime']]]

kf = cross_validation.KFold(train.shape[0], n_folds=3, random_state=1)

pred = []
full_test_predictions = []
for alg, predictors in algorithms:
    alg.fit(train[predictors], train['click'])
    test_predictions=alg.predict_proba(X_test.astype(float))[:,1]
    full_test_predictions.append(test_predictions)
test_predictions = (full_test_predictions[0] + full_test_predictions[1]) / 2

test_predictions[test_predictions <= .5] = 0
test_predictions[test_predictions > .5] = 1
pred.append(test_predictions)

y_predict = np.concatenate(pred, axis=0)



ValueError: Number of features of the model must match the input. Model n_features is 4 and input n_features is 7 

In [66]:
train=pd.concat([X_train,y_train],axis=1)

In [68]:
train.columns

Index(['en_category', 'en_merchant', 'en_countrycode', 'en_browserid',
       'en_devid', 'en_weekday', 'en_daytime', 'click'],
      dtype='object')

In [ ]:
# import packages
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Code starts here
cf=confusion_matrix(y_test,y_predict)
acc=accuracy_score(y_test,y_predict)
precision=precision_score(y_test,y_predict)
recall=recall_score(y_test,y_predict)
f_score=f1_score(y_test,y_predict)


print(cf)
print(acc)
print(precision)
print(recall)
print(f1_score)
print(acc)


# Code starts here